In [1]:
## CELL 1
import findspark
findspark.init()
import pyspark
findspark.find()

from pyspark.sql import SparkSession

In [2]:
from pyspark.sql import SparkSession

# Initialize SparkSession with MongoDB and Excel support
spark = SparkSession.builder \
    .appName("Data Analysis") \
    .config("spark.jars.packages", 
            "com.crealytics:spark-excel_2.12:0.13.7,"
            "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.executor.memoryOverhead", "4g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .config("spark.driver.maxResultSize", "8g") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/realEstateData") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/realEstateData") \
    .getOrCreate()

In [3]:
# Read Excel file
df_layoff_sk = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("dataAddress", "'Sheet1'!A1") \
    .load('tech_layoffs.xlsx')

# Read CSV file
df_housing_sk = spark.read.csv('latest_weekly_housing_market_data_most_recent.tsv000', header=True, sep='\t')

In [5]:
# total numbers of entries in df_housing_sk
df_housing_sk.count()

4308763

In [4]:
# For writing the DataFrame `df_layoff_sk` to MongoDB:
df_layoff_sk.write.format("mongo").mode("overwrite") \
    .option("database", "realEstateData") \
    .option("collection", "layoffData") \
    .save()

# For writing the DataFrame `df_housing_sk` to MongoDB:
df_housing_sk.write.format("mongo").mode("overwrite") \
    .option("database", "realEstateData") \
    .option("collection", "housingMarketData") \
    .save()